# Matching: prompt - canción

## Instalación de librerias

In [1]:
!pip install datasets
!pip install translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


In [2]:
from translate import Translator
import pandas as pd
from google.colab import drive
from transformers import pipeline

## Recuperación del dataset

Es importante notar que, en primer lugar, se debe subir el archivo al ambiente local del collab.

In [5]:
df = pd.read_excel('songs_tagged.xlsx')

In [6]:
df.head(1)

,Unnamed: 0,artist,seq,song,emotions
0,0,Elijah Blake,"No, no_x000D_\nI ain't ever trapped out the ba...",Everyday,['fear']


## Solución

In [3]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=3)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Dado que el modelo originalmente se creo en inglés, se traduce el prompt al idioma correspondiente.

In [4]:
def translate_text(input_text):
    # Crea un objeto Translator y especifica el idioma de origen y destino
    translator = Translator(from_lang="es", to_lang="en")

    # Realiza la traducción
    translation = translator.translate(input_text)

    return translation

### Recibiendo las prompts

In [13]:
import random

def buscar_canciones(emotions):
  # Inicializar un diccionario para almacenar las canciones y artistas asociados a cada emoción
  canciones_artistas_por_emocion = {emocion: {'Canciones': [], 'Artistas': []} for emocion in emotions}

  # Iterar sobre el DataFrame y agregar las canciones y artistas a las listas correspondientes
  for index, row in df.iterrows():
      emocion_actual1 = row[4]
      emocion_actual = emocion_actual1[2:-2]
      if emocion_actual in emotions:
          nombre_cancion = row[3]
          nombre_artista = row[1]
          canciones_artistas_por_emocion[emocion_actual]['Canciones'].append(nombre_cancion)
          canciones_artistas_por_emocion[emocion_actual]['Artistas'].append(nombre_artista)

  # Elegir aleatoriamente un índice de emoción
  emocion_aleatoria = random.choice(emotions)

  # Elegir aleatoriamente un índice de canción para la emoción seleccionada
  indice_cancion_aleatorio = random.randint(0, len(canciones_artistas_por_emocion[emocion_aleatoria]['Canciones']) - 1)

  # Obtener el nombre de la canción y del artista seleccionados aleatoriamente
  cancion_seleccionada = canciones_artistas_por_emocion[emocion_aleatoria]['Canciones'][indice_cancion_aleatorio]
  artista_seleccionado = canciones_artistas_por_emocion[emocion_aleatoria]['Artistas'][indice_cancion_aleatorio]

  return f"Deberias escuchar '{cancion_seleccionada}' - {artista_seleccionado} [{emocion_aleatoria}]"

In [11]:
def recomienda(user_input):
  translated_text = translate_text(user_input)
  result = classifier(translated_text)

  emotions = [result[0][i]['label'] for i in range(3)]
  recomendacion = buscar_canciones(emotions)

  return recomendacion

In [14]:
user_input = ""
while user_input.lower() != "salir":
    user_input = input("Tú: ")
    if user_input != "salir":
      response = recomienda(user_input)
      print(f"Chatbot: {response}")

Tú: Acabo de renunciar a mi trabajo
Chatbot: Deberias escuchar 'The Streets of America' - Bad Religion [sadness]
Tú: Me siento feliz
Chatbot: Deberias escuchar 'How About You?' - Chuck Redd [joy]
Tú: Siento que el día esta nublado y lluvioso
Chatbot: Deberias escuchar 'You Should Have Listened to Al' - Al Stewart [anger]
Tú: salir
